<a href="https://colab.research.google.com/github/scartt/CS505_PA/blob/main/PA4_Sijia_Li.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [50]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [51]:
import csv
import numpy as np
import re
import string
import nltk
from matplotlib import pyplot as plt

In [52]:
train_text = []
train_label = []
with open('/content/drive/MyDrive/Colab Notebooks/sentiment-train.csv', "r") as f:
    reader = csv.reader(f)
    next(reader)
    for line in reader:
        label, text = line
        train_text.append(text)
        train_label.append(label)

In [53]:
test_text = []
test_label = []
with open('/content/drive/MyDrive/Colab Notebooks/sentiment-test.csv', "r") as f:
    reader = csv.reader(f)
    next(reader)
    for line in reader:
        label, text = line
        test_text.append(text)
        test_label.append(label)

In [122]:
test_text[:5]

['@stellargirl I loooooooovvvvvveee my Kindle2. Not that the DX is cool, but the 2 is fantastic in its own right.',
 'Reading my kindle2...  Love it... Lee childs is good read.',
 'Ok, first assesment of the #kindle2 ...it fucking rocks!!!',
 "@kenburbary You'll love your Kindle2. I've had mine for a few months and never looked back. The new big one is huge! No need for remorse! :)",
 "@mikefish  Fair enough. But i have the Kindle2 and I think it's perfect  :)"]

In [123]:
train_text[:5]

['I LOVE @Health4UandPets u guys r the best!! ',
 'im meeting up with one of my besties tonight! Cant wait!!  - GIRL TALK!!',
 '@DaRealSunisaKim Thanks for the Twitter add, Sunisa! I got to meet you once at a HIN show here in the DC area and you were a sweetheart. ',
 'Being sick can be really cheap when it hurts too much to eat real food  Plus, your friends make you soup',
 '@LovesBrooklyn2 he has that effect on everyone ']

## Task 1 & 2 Multinomial NB with CountVectorizer

In [56]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer

In [57]:
vectorizer = CountVectorizer(stop_words="english", max_features = 1000)
vectorizer.fit(train_text)
train_data = vectorizer.transform(train_text)
test_data = vectorizer.transform(test_text)

In [58]:
clf = MultinomialNB()
clf.fit(train_data, train_label)
score = clf.score(test_data, test_label)
print("the Accuracy of the trained classifier is: ", score*100, "%")

the Accuracy of the trained classifier is:  78.27298050139275 %


## Task 3 & 4 Regression classifier with CountVectorizer

In [59]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer

In [60]:
clf2 = LogisticRegression()
clf2.fit(train_data, train_label)
score = clf2.score(test_data, test_label)
print("the Accuracy of the trained classifier is: ", score*100, "%")

the Accuracy of the trained classifier is:  76.6016713091922 %


## Task 5 Model Selection

Multinomial NB classifier is better as it scores higher. When no hyperparameters are tuned a single split into training and testing is fine (no validation set). 

## Task 6 & 7 StratifiedKFold

In [61]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

In [72]:
def trans_data(train_text, test_text, Vectorizer, max_features):
    vectorizer = Vectorizer(stop_words="english", max_features = max_features)
    vectorizer.fit(train_text)
    train_data = vectorizer.transform(train_text)
    test_data = vectorizer.transform(test_text)
    return train_data,test_data

In [94]:
def kfold_valid(train_text, test_text, max_features=1000):
    train_data,test_data = trans_data(train_text, test_text, CountVectorizer, max_features=max_features)
    cv = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)
    scores = cross_val_score(MultinomialNB(), train_data, train_label, scoring='accuracy', cv=cv)
    print('the Accuracy with', max_features, 'of the trained classifier is', np.mean(scores), "%")

In [95]:
kfold_valid(train_text, test_text, max_features=1000)
kfold_valid(train_text, test_text, max_features=2000)
kfold_valid(train_text, test_text, max_features=3000)
kfold_valid(train_text, test_text, max_features=4000)

the Accuracy with 1000 of the trained classifier is 0.7222 %
the Accuracy with 2000 of the trained classifier is 0.73475 %
the Accuracy with 3000 of the trained classifier is 0.7398833333333333 %
the Accuracy with 4000 of the trained classifier is 0.7421833333333334 %


In [98]:
print('the best binary strategy is to set to True, the best max features is 4000.')
print('the Accuracy of the best hyper-parameter combination trained classifier is: 0.74 %')

the best binary strategy is to set to True, the best max features is 4000.
the Accuracy of the best hyper-parameter combination trained classifier is: 0.74 %


## Task 8 & 9 & 10 Word2vec

In [144]:
from gensim.models import Word2Vec
import spacy
nlp = spacy.load('en_core_web_sm')
from spacy.lang.en import English
nlp = English()
from spacy.tokenizer import Tokenizer
import nltk
nltk.download('punkt')
from gensim import models

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [149]:
def tokenize(text_list):
    token_list = []
    for line in text_list:
        sent = nltk.word_tokenize(line)
        res = []
        token_list.append(res)
    return token_list

In [150]:
def trans2vec(data, model):
    vectors = []
    for sent in data:
        word_vectors = []
        for word in sent:
            if word in model.wv:
                word_vectors.append(model.wv[word])
        if len(word_vectors)>0:
            vector = np.mean(word_vectors,axis=0)
        else:
            vector = np.zeros(300)
        vectors.append(vector)
    return vectors

In [151]:
def score(train_text, test_text, model):
    train_data = tokenize(train_text)
    test_data = tokenize(test_text)
    train_vectors = trans2vec(train_data, model)
    test_vectors = trans2vec(test_data, model)
    clf = LogisticRegression(max_iter=1000)
    clf.fit(train_vectors, train_label)
    score = clf.score(test_vectors, test_label)
    print('the Accuracy of the trained classifier is', score)

In [152]:
train_data = tokenize(train_text)
model = models.Word2Vec(size=150, window=10, min_count=5, sg=1, workers=10)
model.build_vocab(train_data)  

In [154]:
score(train_text, test_text, model)

the Accuracy of the trained classifier is 0.49303621169916434


**Question:** Does dense feature representation improve the accuracy of
your logistic regression classifier?

**Your answer here:** Yes. 

## Task 11, 12 & 13 PCA Analysis of Shakepeare's Plays

In [156]:
text_list = []
with open('/content/drive/MyDrive/Colab Notebooks/will_play_text.csv', "r") as f:
    reader = csv.reader(f,delimiter=';')
    for line in reader:
        text_list.append(line)

In [159]:
vocab = []
with open('/content/drive/MyDrive/Colab Notebooks/vocab.txt', "r") as f:
    reader = csv.reader(f,delimiter=';')
    for line in reader:
        vocab.append(line)

In [160]:
play_genres = []
with open('/content/drive/MyDrive/Colab Notebooks/play_genres.csv', "r") as f:
    reader = csv.reader(f,delimiter=';')
    for line in reader:
        play_genres.append(line)

In [161]:
from sklearn.decomposition import PCA

In [225]:
len(text_list)

134035

In [213]:
134035/5

26807.0

In [187]:
text_list[:]

[['1', 'Henry IV', '', '', '', 'ACT I'],
 ['2', 'Henry IV', '', '', '', 'SCENE I. London. The palace.'],
 ['3',
  'Henry IV',
  '',
  '',
  '',
  'Enter KING HENRY, LORD JOHN OF LANCASTER, the EARL of WESTMORELAND, SIR WALTER BLUNT, and others'],
 ['4',
  'Henry IV',
  '1',
  '1.1.1',
  'KING HENRY IV',
  'So shaken as we are, so wan with care,'],
 ['5',
  'Henry IV',
  '1',
  '1.1.2',
  'KING HENRY IV',
  'Find we a time for frighted peace to pant,']]

In [262]:
play_to_text = {}
for data in text_list[:110000]:
  name = data[1]
  talk = data[5]
  if name in play_genres[1]:
    if name not in play_to_text:
      play_to_text[name] = []
      play_to_text[name].append(talk)
    else:
      play_to_text[name].append(talk)

In [264]:
play_text = []
play_names = []
for k,v in play_to_text.items():
    play_text.append(' '.join(v))
    play_names.append(k)

In [263]:
play_to_text

{}

In [232]:
vectorizer = CountVectorizer(stop_words="english", vocabulary = vocab)
vectorizer.fit(play_text)
term_doc = vectorizer.transform(play_text)
print(term_doc.shape)

TypeError: ignored

## CountVectorizer PCA

In [176]:
from sklearn.decomposition import PCA

In [175]:
vectorizer = CountVectorizer(stop_words="english", vocabulary = vocab)
vectorizer.fit(play_text)
term_doc = vectorizer.transform(play_text)
print(term_doc.shape)

TypeError: ignored

## TfidfVectorizer PCA

In [239]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [240]:
vectorizer = TfidfVectorizer(stop_words="english", vocabulary = vocab)
vectorizer.fit(play_text)
term_doc = vectorizer.transform(play_text)
print(term_doc.shape)

TypeError: ignored

## Word-Word Matrix

In [246]:
line_text = []
for i, data in enumerate(text_list[:110000]):
    talk = data[5].lower()
    line_text.append(talk)

In [248]:
vectorizer = CountVectorizer(stop_words="english", vocabulary = vocab)
term_doc = vectorizer.fit_transform(line_text)

TypeError: ignored

## Plays Cosine Similarity

In [252]:
from sklearn.metrics.pairwise import cosine_similarity
from itertools import combinations